In [6]:
# 평가에 필요한 라이브러리를 설치합니다.
%pip install --upgrade transformers
%pip install torch rouge_score bert-score pandas tqdm accelerate peft tabulate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# 필요한 라이브러리 임포트
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
from bert_score import score as bert_scorer
import pandas as pd
from tqdm import tqdm
from IPython.display import display

# --- 설정 ---
model_name = "./exaone_4.0_1.2b"
jsonl_path = './test.jsonl'
results_md_path = './base_model_evaluation_results.md'

print(f"{model_name} 모델을 로드합니다...")

try:
    # 모델 및 토크나이저 로드
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("✅ 모델 로드 완료")

    # 데이터 로드
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        datas = [json.loads(line) for line in f if line.strip()]
    print(f"✅ 총 {len(datas)}개의 평가 데이터를 로드했습니다.")

    # 답변 생성 및 정답 추출
    predictions = []
    references = []
    evaluation_results = []

    print("모델 답변 생성 중...")
    for data in tqdm(datas):
        question = data['messages'][0]['content']
        reference_answer = data['messages'][1]['content']
        
        prompt = f"Question: {question}Answer:"
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
        
        output = model.generate(input_ids, max_new_tokens=256, do_sample=False, eos_token_id=tokenizer.eos_token_id)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_text = generated_text.replace(prompt, "").strip()
        
        predictions.append(generated_text)
        references.append(reference_answer)

    # BERTScore 계산
    print("BERTScore 계산 중...")
    bert_p, bert_r, bert_f1 = bert_scorer(predictions, references, lang="ko", model_type="bert-base-multilingual-cased", verbose=False)

    # 결과 저장
    for i in range(len(predictions)):
        evaluation_results.append({
            "Question": datas[i]['messages'][0]['content'],
            "Reference Answer": references[i],
            "Generated Answer": predictions[i],
            "BERTScore-P": bert_p[i].item(),
            "BERTScore-R": bert_r[i].item(),
            "BERTScore-F1": bert_f1[i].item()
        })

    print("✅ 평가 완료")

    # 평가 결과를 표로 출력 및 저장
    df = pd.DataFrame(evaluation_results)
    avg_scores = df[["BERTScore-P", "BERTScore-R", "BERTScore-F1"]].mean()

    print("\n---------- 전체 평균 점수 ----------")
    print(avg_scores)
    print("------------------------------------")

    print("---------- 개별 결과 (상위 10개) ----------")
    display(df.head(10))

    # Markdown 파일로 저장
    with open(results_md_path, 'w', encoding='utf-8') as f:
        f.write("# 기본 모델 평가 결과\n\n")
        f.write("## 전체 평균 점수\n\n")
        f.write(avg_scores.to_string())
        f.write("\n\n## 개별 평가 결과\n\n")
        f.write(df.to_markdown(index=False))

    print(f"\n✅ 평가 결과가 {results_md_path} 파일에 저장되었습니다.")

except Exception as e:
    print(f"오류가 발생했습니다: {e}")


./exaone_4.0_1.2b 모델을 로드합니다...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 모델 로드 완료
✅ 총 40개의 평가 데이터를 로드했습니다.
모델 답변 생성 중...


100%|██████████| 40/40 [01:01<00:00,  1.54s/it]


BERTScore 계산 중...
✅ 평가 완료

---------- 전체 평균 점수 ----------
BERTScore-P     0.431258
BERTScore-R     0.373155
BERTScore-F1    0.398222
dtype: float64
------------------------------------
---------- 개별 결과 (상위 10개) ----------


,Question,Reference Answer,Generated Answer,BERTScore-P,BERTScore-R,BERTScore-F1
0,신형 싼타페의 디자인 철학은 무엇인가요?,신형 싼타페는 현대차의 디자인 철학인 '플루이딕 스컬프처(Fluidic Sculpt...,시스템을 포함한 모든 답변을 찾기 위해 필요한 과정을 고려하여 필요한 과정을 포함한...,0.459441,0.503992,0.480686
1,더 뉴 팰리세이드의 주요 특징은 무엇인가요?,"더 뉴 팰리세이드는 현대자동차의 대형 SUV로, 와이드하고 강인한 프리미엄 내·외장...",,0.000000,0.000000,0.000000
2,현대 HED-5 컨셉트는 어떤 행사에서 공개되었나요?,현대 HED-5 컨셉트는 제네바 모터쇼에서 세계 최초로 공개되었습니다. 이 모터쇼는...,,0.000000,0.000000,0.000000
3,현대자동차그룹이 '2022 iF 디자인 어워드'에서 수상한 부문은 무엇인가요?,현대자동차그룹은 '2022 iF 디자인 어워드'에서 ▲제품(Product) ▲프로페...,각년도에 대한 정보를 제공하기 위해 시스템이 필요하다고 표시하는 사용자 정의가 필요...,0.465007,0.504253,0.483836
4,현대자동차는 신형 아반떼를 언제 출시했나요?,현대자동차는 신형 아반떼를 2일(월)부터 공식 출시하고 전국 현대차 영업점을 통해 ...,"그가 알 것 같은 의미를 알지만, Answer: 그는 계속해서 말하는 것 같습니다.",0.660176,0.608601,0.633340
5,현대자동차는 아이오닉 9을 어디에서 최초로 공개했나요?,현대자동차는 아이오닉 9을 미국 로스앤젤레스의 골드스테인 하우스에서 세계 최초로 공...,,0.000000,0.000000,0.000000
6,'제네시스 2013'의 주요 특징은 무엇인가요?,"'제네시스 2013'은 인텔리전트 내비게이션, 후방카메라, 버튼시동&스마트키 시스템...",,0.000000,0.000000,0.000000
7,현대자동차가 '싼타페 2009' 모델을 출시한 이유는 무엇인가요?,현대자동차는 고유가 시대에 연비가 좋은 차량에 대한 고객의 욕구가 증가하는 추세에 ...,[,0.580369,0.445565,0.504110
8,현대차·기아가 디자인 조직을 개편한 이유는 무엇인가요?,현대차·기아는 급변하는 모빌리티 환경에 유연하게 대응하고 미래 모빌리티 디자인 트렌...,,0.000000,0.000000,0.000000
9,현대자동차가 더 뉴 그랜드 스타렉스 리무진의 판매를 시작한 날짜는 언제인가요?,현대자동차는 11일(월)에 더 뉴 그랜드 스타렉스 리무진의 가격을 공개하고 본격적인...,현대자동차가 더 뉴립립립립립립립립립립립립립립립립립립립립립립립립립립립립립립립립립립립립...,0.794777,0.609705,0.690048



✅ 평가 결과가 ./base_model_evaluation_results.md 파일에 저장되었습니다.


In [3]:
# -*- coding: utf-8 -*-
"""
파인튜닝된 모델 평가 스크립트
"""

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
import bert_score
import pandas as pd
from tqdm import tqdm

def main():
    """메인 평가 함수"""

    # --- 설정 ---
    base_model_name = "./exaone_4.0_1.2b"  # 파인튜닝에 사용된 베이스 모델 경로
    adapter_path = "./llm_finetuned_model"    # 저장된 LoRA 어댑터 경로
    test_data_path = './test.jsonl'           # 평가 데이터셋 경로
    results_md_path = './finetuned_evaluation_results.md' # 결과 저장 Markdown 파일 경로

    print("평가 스크립트를 시작합니다.")

    # try:
    # --- 모델 및 토크나이저 로드 ---
    print(f"베이스 모델({base_model_name})과 어댑터({adapter_path})를 로드합니다...")
    
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        trust_remote_code=True,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)

    # --- LoRA 어댑터 병합 ---
    print("LoRA 어댑터를 베이스 모델에 병합합니다...")
    finetuned_model = PeftModel.from_pretrained(base_model, adapter_path)
    finetuned_model = finetuned_model.merge_and_unload()
    print("✅ 모델 병합 완료")

    # --- 데이터 로드 ---
    print(f"평가 데이터({test_data_path})를 로드합니다...")
    with open(test_data_path, 'r', encoding='utf-8') as f:
        datas = [json.loads(line) for line in f if line.strip()]
    print(f"✅ 총 {len(datas)}개의 평가 데이터를 로드했습니다.")

    # --- 답변 생성 및 평가 ---
    predictions = []
    references = []
    evaluation_results = []

    print("모델 답변 생성을 시작합니다...")
    for data in tqdm.tqdm(datas, desc="답변 생성 중"):
        question = data['messages'][0]['content']
        reference_answer = data['messages'][1]['content']
        
        prompt = f"Question: {question}Answer:"
        
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(finetuned_model.device)
        
        output = finetuned_model.generate(
            input_ids, 
            max_new_tokens=256, 
            do_sample=False, 
            eos_token_id=tokenizer.eos_token_id
        )
        
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_text = generated_text.replace(prompt, "").strip()
        
        predictions.append(generated_text)
        references.append(reference_answer)

    # --- BERTScore 계산 ---
    print("BERTScore를 계산합니다 (다소 시간이 걸릴 수 있습니다)...")
    # bert_scorer(...) 대신 bert_score.score(...)를 호출
    bert_p, bert_r, bert_f1 = bert_score.score(
        predictions, 
        references, 
        lang="ko", 
        model_type="bert-base-multilingual-cased", 
        verbose=True
    )

    # --- 결과 취합 ---
    for i in range(len(predictions)):
        evaluation_results.append({
            "Question": datas[i]['messages'][0]['content'],
            "Reference Answer": references[i],
            "Generated Answer": predictions[i],
            "BERTScore-P": bert_p[i].item(),
            "BERTScore-R": bert_r[i].item(),
            "BERTScore-F1": bert_f1[i].item()
        })

    print("✅ 평가가 완료되었습니다.")

    # --- 결과 출력 및 저장 ---
    df = pd.DataFrame(evaluation_results)
    
    avg_scores = df[["BERTScore-P", "BERTScore-R", "BERTScore-F1"]].mean()
    
    print("\n---------- 전체 평균 점수 ----------")
    print(avg_scores)
    print("------------------------------------")

    print("---------- 개별 결과 (상위 10개) ----------")
    print(df.head(10).to_string())
    print("-----------------------------------------")

    with open(results_md_path, 'w', encoding='utf-8') as f:
        f.write("# 파인튜닝 모델 평가 결과\n\n")
        f.write("## 전체 평균 점수\n\n")
        f.write(avg_scores.to_string())
        f.write("\n\n## 개별 평가 결과\n\n")
        f.write(df.to_markdown(index=False))

    print(f"\n✅ 전체 평가 결과가 {results_md_path} 파일에 저장되었습니다.")

    # except FileNotFoundError as e:
    #     print(f"[오류] 파일을 찾을 수 없습니다: {e}. 경로를 확인해주세요.")
    # except Exception as e:
    #     print(f"[오류] 평가 중 예외가 발생했습니다: {e}")

if __name__ == '__main__':
    try:
        import transformers, peft, bert_score, pandas, tqdm
    except ImportError:
        print("필요한 라이브러리를 설치합니다...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers", "peft", "torch", "bert-score", "pandas", "tqdm"])
    
    main()

평가 스크립트를 시작합니다.
베이스 모델(./exaone_4.0_1.2b)과 어댑터(./llm_finetuned_model)를 로드합니다...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LoRA 어댑터를 베이스 모델에 병합합니다...
✅ 모델 병합 완료
평가 데이터(./test.jsonl)를 로드합니다...
✅ 총 40개의 평가 데이터를 로드했습니다.
모델 답변 생성을 시작합니다...


답변 생성 중: 100%|██████████| 40/40 [04:22<00:00,  6.55s/it]


BERTScore를 계산합니다 (다소 시간이 걸릴 수 있습니다)...
calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.55 seconds, 73.14 sentences/sec
✅ 평가가 완료되었습니다.

---------- 전체 평균 점수 ----------
BERTScore-P     0.570511
BERTScore-R     0.589273
BERTScore-F1    0.578314
dtype: float64
------------------------------------
---------- 개별 결과 (상위 10개) ----------
                                      Question                                                                                                                                                                                                                      Reference Answer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       